**Model 1 **: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

**Model 2:** https://huggingface.co/google/flan-t5-xxl

Metric used for Evaluation : Semantic Answer Similarity (SAS)

Semantic Answer Similarity (SAS): https://arxiv.org/abs/2108.06130

In [ ]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=3dbd59ff53b33e30f04ec96d27205958a52a3ddbfe7ba31e9c1d5eaefa17604a
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util

#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-distilbert-dot-v1')

26.777278900146484 atoms are the size of a molecule
16.000530242919922 atomic
12.464305877685547 proportional


In [ ]:
#------------Sample Implementation for Sentence Similarity -----------------

#Initialize the inputs
#query = "Atoms are the building blocks that come together through chemical bonding to form what in the universe?"
#answers = ["Molecule", "dummy"]
#
#query = "How do atoms come together?"
#answers = ["chemical bonding", "when they are bonded together"]
#
#query = "What is the model of atoms in the universe?"
#answers = ["molecule", "Carbon (black), hydrogen (white), nitrogen (blue), oxygen (red) and sulfur" , "atoms are made of protons and neutrons"]

query = "What size are the atoms in the model of a molecule?"
answers = ["atoms are the size of a molecule","atomic","proportional"]

#Encode query and documents
query_emb = model.encode(query)
doc_emb = model.encode(answers)

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(answers, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

#Output passages & scores
for doc, score in doc_score_pairs:
    print(score, doc)

In [ ]:
# connect your personal google drive to store dataset and trained model
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
# Read the excel which contains the context, original keyword, generated question and the predicted answer
import pandas as pd
source=pd.read_excel("/content/gdrive/My Drive/CS 677 Project/dataset/predicted_answer.xlsx", sheet_name='Sheet_name_1')

In [ ]:
# Spot check the data
source.head()

,Unnamed: 0,0,1,2,3
0,0,Atoms are the building blocks that come togeth...,form molecules,Atoms are the building blocks that come toget...,form molecules
1,1,Atoms are the building blocks that come togeth...,chemical bonding,How do atoms come together?,Chemical bonding
2,2,Atoms are the building blocks that come togeth...,building blocks,What are atoms?,Building blocks
3,3,Atoms are the building blocks that come togeth...,Atoms,What are the building blocks that come togeth...,Atoms
4,4,Atoms are the building blocks that come togeth...,Christian Guthier,Who modified the model of atoms?,Christian Guthier


In [ ]:
# Function to calculate similarity score between the original keyword and the predicted answer

def get_similarity_scores(question, answers):
  #Encode query and documents
  query_emb = model.encode(question)
  answer_emb = model.encode(answers)
  #Compute dot score between query and all document embeddings
  scores = util.dot_score(query_emb, answer_emb)[0].cpu().tolist()
  return scores

In [ ]:
# Function to compare two similarity scores based on a threshold
threshold = 10

def check_similarity(score1, score2):
  if abs(score1-score2) <=threshold :
    return 'Match'
  else:
    return 'No Match'

In [ ]:
# Spot check the function using an example
similarity_scores = get_similarity_scores('Atoms are the building blocks that come together through chemical bonding to do what in the universe?', 
                 ['form molecules','ten form molecules'])

check_similarity(similarity_scores[0], similarity_scores[1])

In [ ]:
# Collect Valid QA pairs

QA_pairs = []
for index, row in source.iterrows():
  context = row[0] 
  original_keyword = row[1]
  question = row[2]
  predicted_answer = row[3]
  answers = [original_keyword, predicted_answer]
  similarity_scores = get_similarity_scores(question, answers)
  check = check_similarity(similarity_scores[0], similarity_scores[1])
  if check == 'Match':
    valid_QA_tuple = (context, question, predicted_answer, similarity_scores[1] )
    QA_pairs.append(valid_QA_tuple)

In [ ]:
# Spot check valid QA pair
QA_pairs[:2]

[('Atoms are the building blocks that come together through chemical bonding to form molecules in the universe. In this model of a molecule, the atoms of carbon (black), hydrogen (white), nitrogen (blue), oxygen (red), and sulfur (yellow) are in proportional atomic size. The silver rods indicate chemical bonds that hold the atoms together in a specific three-dimensional shape. (credit: modification of work by Christian Guthier).\nThe elements carbon, hydrogen, nitrogen, oxygen, sulfur, and phosphorus are the key building blocks found in all living things. ',
  ' Atoms are the building blocks that come together through chemical bonding to do what in the universe?',
  'form molecules',
  19.808414459228516),
 ('Atoms are the building blocks that come together through chemical bonding to form molecules in the universe. In this model of a molecule, the atoms of carbon (black), hydrogen (white), nitrogen (blue), oxygen (red), and sulfur (yellow) are in proportional atomic size. The silver r

In [ ]:
# Select only distinct QA pairs
dist_QA_pairs = list(set(QA_pairs))

# Spot check distinct QA pair
dist_QA_pairs[:2]

[("For example, one gold atom has all the properties of gold, such as being a solid metal at room temperature. A gold coin is simply a very large number of gold atoms molded into the shape of a coin and contains small amounts of other elements known as impurities. We cannot break gold atoms down into anything smaller while still retaining the properties of gold.\nAn atom is composed of two regions. The center of the atom, which is called the nucleus, contains subatomic particles called protons and neutrons. The atom's outermost region holds subatomic particles known as electrons. ",
  ' What type of particles are in the center of an atom?',
  'Protons and neutrons',
  17.009929656982422),
 ("The atom will have no charge because the positive and negative charges cancel each other out.\nMost of an atom's volume, greater than 99 percent, is empty space. With all this empty space, one might ask why solid objects do not just pass through one another. The reason this does not occur is due to

In [ ]:
# Write the final output to excel

final_QA_pairs = pd.DataFrame(dist_QA_pairs)
final_QA_pairs.to_excel("/content/gdrive/My Drive/CS 677 Project/dataset/final_QA_pairs.xlsx", sheet_name='Sheet_name_1') 